Use `uproot` can read ROOT objects from root type file without relying on ROOT I/O library

In [1]:
import uproot
import json
from io import StringIO

import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_addons as tfa
#from keras.layers.core import MaxoutDense

from models.maxout_layers import Maxout1D
from models.normalization_layer import Normalization


def get_net_struct(obj_path):
    '''
    Directly read ROOT objects specified in obj_path.
    obj_path -- file.root:Tdiectory/subdirectory/..../obj
    
    In this fuction we only need TString obj from the file.
    '''
    with uproot.open(File_path) as net_config:
    #convert string to dictionary
        assert (type(net_config) is uproot.models.TObjString.Model_TObjString )
        _struct = json.load(StringIO(net_config))
    return _struct

def print_dimention(weights):
    for w in weights[1:]:
        print(w["weights"])

def all_layers(weights):
    for i, w in enumerate(weights):
        print(i, w.keys())
        
def get_maxout_weights(NN_layer):
    maxout_unit=0
    maxout_h_unit=len(NN_layer['sublayers'][maxout_unit]['bias'])
    in_features = len(NN_layer['sublayers'][maxout_unit]['weights'])//maxout_h_unit
    maxout_weights=[]
    maxout_biases = []
    units = len(NN_layer['sublayers'])
    
    for maxout_unit in range(units):
        maxout_weights.append(
                                np.array(NN_layer['sublayers'][maxout_unit]['weights']
                              ).reshape( maxout_h_unit, in_features).transpose() )
        maxout_biases.append(
                                np.array(NN_layer['sublayers'][maxout_unit]['bias'])
                            )
    
    return (in_features, maxout_h_unit, units, 
            np.concatenate(maxout_weights, axis=-1),
            np.concatenate(maxout_biases, axis=-1) )
   

def get_dense_weights(NN_layer):
    h_unit=len(NN_layer["bias"])
    in_features = len(NN_layer['weights'])//h_unit
    weight = np.array(NN_layer['weights']).reshape( h_unit, in_features).transpose()
    return (in_features, h_unit, weight, np.array(NN_layer["bias"]) )

def get_BN_weights(NN_layer):
    h_unit=len(NN_layer["bias"])
    return (np.diag(np.array(NN_layer['weights'])),
            np.array(NN_layer["bias"]) )


def pars_layers(layers):
    N_layers = len(layers)
    layersDic = {}
    tf_layers = []
    N_features = -1
    for i, layer in enumerate(layers):
        arch = layer["architecture"]
        if arch == 'maxout':
            layer_name="maxout%s"%i
            
            # return Nfeatures, hiden nodes, maxout units, weights, bias
            v, h,unit, w, b = get_maxout_weights(layer) 
            if N_features<1:  N_features = v
                
            layersDic[layer_name] = [w, b]
            tf_layers.append( Maxout1D(h, unit, name=layer_name) )
            #tf_layers.append( keras.layers.Activation(
            #                                            activation=layer["activation"],
            #                                            name="activ%s"%i 
            #                                            )
            #                )
            
        elif arch == 'normalization':
            layer_name="BN%s"%i 
            layersDic[layer_name] = [*get_BN_weights(layer)]
            tf_layers.append( Normalization(name=layer_name) )
            
        elif arch == 'dense':
            layer_name="dense%s"%i
            #Ninputs, hiden nodes, weights, bias
            v, h, w, b = get_dense_weights(layer)
            if N_features<1: N_features = v
            layersDic[layer_name]=[w, b ]
            activation="relu" if layer["activation"]=='rectified' else layer["activation"]
            
            tf_layers.append( keras.layers.Dense(h, activation=activation,
                              kernel_initializer='glorot_uniform', name=layer_name)
                            )
        else:
            raise Exception('Unkown layer %s'%arch )
            
    return N_features, tf_layers, layersDic

#create NN from input layers
#each layer has unique name
def get_DL1(N_features, dl1_layers, lr=0.005, drops=None):
    
    In = tf.keras.layers.Input(shape=(N_features,), name="input")
    x = In
    drop_index=0
    for layer in dl1_layers[:-1]:
        if drops:
            if 'BN' in layer.name:
                x = keras.layers.Dropout( drops[drop_index], 
                                          name="drop%s"%drop_index )(x, training=True)
                drop_index=drop_index+1
        x = layer(x) 
        
    predictions = dl1_layers[-1](x)
    
    model = keras.models.Model(inputs=In, outputs=predictions)
    model_optimizer = keras.optimizers.Adam(lr=lr)
    model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
        optimizer=model_optimizer,
        metrics=['accuracy']
    )
    return model

#uproot.open("BTagCalibRUN2-08-40.root").keys()

Interested Dl1 networkes
* 'DL1',
* 'DL1/AntiKt4EMTopo',
* 'DL1/AntiKt4EMTopo/net_configuration',
* 'DL1mu',
* 'DL1mu/AntiKt4EMTopo',
* 'DL1mu/AntiKt4EMTopo/net_configuration',
* 'DL1rnn',
* 'DL1rnn/AntiKt4EMTopo',
* 'DL1rnn/AntiKt4EMTopo/net_configuration',


## DL1   
DL1 is a neural network trained by b-tagging group. 
Model weights are stored in `BTagCalibRUN2-08-40.root` file as a string object. Our goal is to read the weight strings and convert them into json format.

In [22]:
#filename.root:Tdirectory/directory/obj
File_path="BTagCalibRUN2-08-40.root:DL1/AntiKt4EMTopo/net_configuration"

DL1_struct = get_net_struct(File_path)
DL1_weights = DL1_struct['layers']

print(f"number of layers: {len(DL1_weights)}")
all_layers(DL1_weights)
#print_dimention(weights)
print(DL1_weights[2]['weights'])

number of layers: 17
0 dict_keys(['sublayers', 'activation', 'architecture'])
1 dict_keys(['bias', 'weights', 'architecture'])
2 dict_keys(['bias', 'weights', 'activation', 'architecture'])
3 dict_keys(['bias', 'weights', 'architecture'])
4 dict_keys(['bias', 'weights', 'activation', 'architecture'])
5 dict_keys(['bias', 'weights', 'architecture'])
6 dict_keys(['bias', 'weights', 'activation', 'architecture'])
7 dict_keys(['bias', 'weights', 'architecture'])
8 dict_keys(['bias', 'weights', 'activation', 'architecture'])
9 dict_keys(['bias', 'weights', 'architecture'])
10 dict_keys(['sublayers', 'activation', 'architecture'])
11 dict_keys(['bias', 'weights', 'architecture'])
12 dict_keys(['bias', 'weights', 'activation', 'architecture'])
13 dict_keys(['bias', 'weights', 'architecture'])
14 dict_keys(['bias', 'weights', 'activation', 'architecture'])
15 dict_keys(['bias', 'weights', 'architecture'])
16 dict_keys(['bias', 'weights', 'activation', 'architecture'])
[-0.002210443839430809, -

The `sublayers` stored weights and biases of MaxoutDense layers.  Total of two MaxoutDense leyers are stored.  
Other layers are BatchNoramlization and Dense layers.  

In [3]:
# quick check maxout layers
sublayers = DL1_struct['layers'][0]['sublayers']

#output size of a maxout layer
print(len(sublayers[0]['bias']))

#last Dense layer (output layer)
DL1_struct['layers'][-5]['activation']

72


'rectified'

In [4]:

DL1_struct['layers'][-3]['activation']


'rectified'

In the bellow, I defin DL1rnn with tensorflow keras API. Instead of train the new network, I will set weights of each layer to the weights extracted above. 

In [5]:
#DL1_layers = [ 72, 57, 60, 48, 36,24, 12, 6]
DL1_dropouts = [0.1, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]
dropout_enable = False

#findout input size, and weight matrix for each layer.
#create corresponding tensorflow layers and store them in a list.
features, dl1_layers, dl1_weights = pars_layers(DL1_struct['layers'])

DL1_model = get_DL1(features , dl1_layers, drops=DL1_dropouts if dropout_enable else None )
DL1_model.summary()

def set_dl1_weights(model, weights):
    for name in weights.keys():
        print(name)
        layer = model.get_layer( name=name)
        layer.set_weights(weights[name])
        
set_dl1_weights(model=DL1_model, weights=dl1_weights)

(None, 25, 72)
(None, 25, 24)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 41)]              0         
_________________________________________________________________
maxout0 (Maxout1D)           (None, 72)                75600     
_________________________________________________________________
BN1 (Normalization)          (None, 72)                5256      
_________________________________________________________________
dense2 (Dense)               (None, 57)                4161      
_________________________________________________________________
BN3 (Normalization)          (None, 57)                3306      
_________________________________________________________________
dense4 (Dense)               (None, 60)                3480      
_________________________________________________________________
BN5 (Normalization)          (N

## test the model with a random inputs

In [6]:
#test model with dummy inputs
DL1_model(inputs=np.random.random((5, features)), training=False)
#DL1_model.layers[3].weights

(5, 25, 72)
(5, 25, 24)


<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[1.4432084e-08, 3.7255999e-02, 9.6274400e-01],
       [1.2950706e-08, 3.6354259e-02, 9.6364570e-01],
       [2.5261575e-09, 2.5060263e-02, 9.7493976e-01],
       [7.8785733e-09, 3.2477390e-02, 9.6752262e-01],
       [3.6765939e-09, 2.7302764e-02, 9.7269720e-01]], dtype=float32)>

## Save this model

This model contains a custom layer which can not be saved as a single `.h5` file with `save("model.h5")`. Becuase, the custom layer implemented in the model is not know, and you will get an error when loading the model again.   
Alternatively, `save("DL1_AntiKt4EMTopo")` will save our model into a directory which contains model architecture and weights.



In [7]:
model_file = "DL1_AntiKt4EMTopo_dropout" if dropout_enable else "DL1_AntiKt4EMTopo"
DL1_model.save(model_file)

(None, 25, 72)
(None, 25, 24)
(None, 25, 72)
(None, 25, 24)
(None, 25, 72)
(None, 25, 24)
(None, 25, 72)
(None, 25, 24)
(None, 25, 72)
(None, 25, 24)
INFO:tensorflow:Assets written to: DL1_AntiKt4EMTopo/assets


## Load model

load_model() fuction `tf.keras.models.load_model("DL1_AntiKt4EMTopo")` can directly load model architectures and weights including the custom layer.

In [8]:
test_model = tf.keras.models.load_model(model_file)
test_model.summary()

AssertionError: Some Python objects were not bound to checkpointed values, likely due to changes in the Python program: [<tf.Variable 'mean:0' shape=(57,) dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.], dtype=float32)>, <tf.Variable 'count:0' shape=() dtype=int64, numpy=0>, <tf.Variable 'mean:0' shape=(6,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0.], dtype=float32)>, <tf.Variable 'mean:0' shape=(24,) dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.], dtype=float32)>, <tf.Variable 'variance:0' shape=(36,) dtype=float32, numpy=
array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1

In [ ]:
test_model(inputs=np.random.random((5, features)), training=False)

In [ ]:
DL1_weights[0]['sublayers'][0]['weights'][:41]

In [ ]:
dl1_weights['maxout0'][0][:,0].flatten()

In [ ]:
a = tf.constant([1, 2, 3, 4, 5, 6], shape=[2, 3])
b = tf.constant([7, 9, 11], shape=[3,1])

In [ ]:
tf.linalg.matvec(a, b)

In [11]:
import numpy as np
mat1=np.array([[1,1,1], [2,2,2],[3,3,3]])
mat2=np.array([[4,4,4], [5,5,5],[6,6,6]])
np.concatenate([mat1[0], mat2[0]], axis=-1)

array([1, 1, 1, 4, 4, 4])

In [12]:
import tensorflow as tf
import tensorflow_addons as tfa
t1=[1,2,3,4,5,6]
tf.reshape(t1, [2,3])

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]], dtype=int32)>

In [13]:
tfa.layers.Maxout(1,0)(tf.reshape(t1, [2,3]))

<tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[4, 5, 6]], dtype=int32)>

In [19]:
m=np.array([1,2,3,4,5,6]).reshape(3,2).T.flatten()

In [20]:
m.reshape(2,3).T

array([[1, 2],
       [3, 4],
       [5, 6]])